In [1]:
import numpy as np
import nibabel as nib

import dipy.tracking.utils
import dipy.tracking.streamline

import pandas as pd
import os

import nipype.interfaces.dipy
import nipype.interfaces.mrtrix3
import nipype.interfaces.fsl

In [88]:
patient_id = 1

stimulation_site = "s00"
recording_electrode = "e01"

chunk_id = 2

In [84]:
data_dir = "/user/pfilipia/home/inria/chu_nice_inria/patients_dmri"

def get_patient_dir(patient_id):
    return os.path.join(
        data_dir, "patient%02d/bids/sub-patient%02d/ses-presurgical" % (patient_id, patient_id)
    )

def get_connectivity_dir(patient_id):
    return os.path.join(
        get_patient_dir(patient_id), "connectivity"
    )

def get_tck_streamlines_dir(patient_id):
    return os.path.join(
        get_connectivity_dir(patient_id), "tck_streamlines"
    )

def get_tck_streamlines_file(patient_id, stimulation_site, recording_electrode):
    return os.path.join(
        get_tck_streamlines_dir(patient_id),
        "%s_%s_diam10_in00.tck" % (stimulation_site, recording_electrode)
    )

def get_single_tck_streamline_file(patient_id, stimulation_site, recording_electrode, streamline_id):
    return os.path.join(
        get_tck_streamlines_dir(patient_id),
        "%s_%s_%09d.tck" % (stimulation_site, recording_electrode, streamline_id)
    )    

def get_trk_streamlines_dir(patient_id):
    return os.path.join(
        get_connectivity_dir(patient_id), "streamlines"
    )

def get_trk_streamlines_file(patient_id, stimulation_site, recording_electrode):
    return os.path.join(
        get_trk_streamlines_dir(patient_id),
        "%s_%s_diam10_in00.trk" % (stimulation_site, recording_electrode)
    )

def get_weights_file(patient_id, stimulation_site, recording_electrode):
    return os.path.join(
        get_tck_streamlines_dir(patient_id),
        "%s_%s_diam10_in00_weight.txt" % (stimulation_site, recording_electrode)
    )

def get_single_weight_file(patient_id, stimulation_site, recording_electrode, streamline_id):
    return os.path.join(
        get_tck_streamlines_dir(patient_id), 
        "%s_%s_%09d_weight.txt" % (stimulation_site, recording_electrode, streamline_id)
    )        

def get_tdi_file(patient_id, stimulation_site, recording_electrode):
    return os.path.join(
        get_tck_streamlines_dir(patient_id), 
        "%s_%s_diam10_in00_weighted_tckmap.nii.gz" % (stimulation_site, recording_electrode)
    )

def get_single_tdi_file(patient_id, stimulation_site, recording_electrode, streamline_id):
    return os.path.join(
        get_tck_streamlines_dir(patient_id), 
        "%s_%s_%09d_tdi.nii" % (stimulation_site, recording_electrode, streamline_id)
    )    

def get_tractography_dir(patient_id):
    return os.path.join(
        get_patient_dir(patient_id), "tractography"
    )

def get_dti_measure_file(patient_id, measure_name):
    return os.path.join(
        get_tractography_dir(patient_id), 
        "sub-patient%02d_ses-presurgical_dti_%s.nii.gz" % (patient_id, measure_name)
    )

def get_fa_file(patient_id):
    return get_dti_measure_file(patient_id, "FA")

def get_md_file(patient_id):
    return get_dti_measure_file(patient_id, "MD")
    
def get_ad_file(patient_id):
    return get_dti_measure_file(patient_id, "L1")

def get_l2_file(patient_id):
    return get_dti_measure_file(patient_id, "L2")

def get_l3_file(patient_id):
    return get_dti_measure_file(patient_id, "L3")
    
def get_template_file(patient_id):
    return get_fa_file(patient_id)
    
def get_microstructure_indices_dir(patient_id):
    return os.path.join(
        get_tck_streamlines_dir(patient_id), "indices"
    )
    
def get_microstructure_index_file(patient_id, index_name):
    return os.path.join(
        get_microstructure_indices_dir(patient_id), 
        "patient%02d_roi_%s.nii.gz" % (patient_id, index_name)
    )

def get_rtop_file(patient_id):
    return get_microstructure_index_file(patient_id, "rtop")

def get_rtap_file(patient_id):
    return get_microstructure_index_file(patient_id, "rtap")

def get_rtpp_file(patient_id):
    return get_microstructure_index_file(patient_id, "rtpp")

def get_msd_file(patient_id):
    return get_microstructure_index_file(patient_id, "msd")

def get_qiv_file(patient_id):
    return get_microstructure_index_file(patient_id, "qiv_e9")

def get_ng_file(patient_id):
    return get_microstructure_index_file(patient_id, "ng")

def get_ng_perp_file(patient_id):
    return get_microstructure_index_file(patient_id, "ng_perp")

def get_ng_par_file(patient_id):
    return get_microstructure_index_file(patient_id, "ng_par")

def get_average_indices_file(patient_id, stimulation_site, recording_electrode, chunk_id):
    return os.path.join(
        get_tck_streamlines_dir(patient_id), 
        "%s_%s_indices_%d.csv" % (stimulation_site, recording_electrode, chunk_id)
    )    

In [4]:
def delete_file_if_exists(file_name):
    if os.path.exists(file_name):
        os.remove(file_name)
    

In [6]:
tck_file = nib.streamlines.load(get_tck_streamlines_file(patient_id, stimulation_site, recording_electrode))
streamline_list = tck_file.streamlines
streamlines_lengths = list(dipy.tracking.utils.length(streamline_list))
    
f = open(get_weights_file(patient_id, stimulation_site, recording_electrode), 'r')
streamline_weights = f.readline().split(" ")
f.close()


In [89]:
single_index_data = {}

all_indices = [
    'fa', 'md', 'ad', 'rd',
    'rtop', 'rtap', 'rtpp', 'msd', 'qiv_e9', 'ng', 'ng_perp', 'ng_par'
]

f = open(get_average_indices_file(patient_id, stimulation_site, recording_electrode, chunk_id), 'w')
f.write("weight\t")
for single_index in all_indices:
    f.write("%s\t" % single_index)
f.write("\n")
f.close()

nib_template = nib.load(get_template_file(patient_id))

for dti_index in ['fa', 'md', 'ad', 'rd']:

    single_index_data[dti_index] = nib.load(
        get_dti_measure_file(patient_id, dti_index.upper())
    ).get_data()

for microstructure_index in ['rtop', 'rtap', 'rtpp', 'msd', 'qiv_e9', 'ng', 'ng_perp', 'ng_par']:

    single_index_data[microstructure_index] = nib.load(
        get_microstructure_index_file(patient_id, microstructure_index)
    ).get_data()


for i in range(30): # range(len(streamline_list)):

    if np.mod(i, 10) != chunk_id:
        continue
    
    if len(streamlines_lengths) <= 0:
        continue

    single_index_means = {}

    try:
        single_tdi_data = dipy.tracking.utils.density_map(
            [streamline_list[i]], nib_template.shape, affine=nib_template.affine
        )

        for single_index in all_indices:
            single_index_means[single_index] = np.mean(single_index_data[single_index][single_tdi_data.nonzero()])

        f = open(get_average_indices_file(patient_id, stimulation_site, recording_electrode, chunk_id), 'a')
        f.write("%s\t" % streamline_weights[i])
        for single_index in all_indices:
            f.write("%f\t" % single_index_means[single_index])
        f.write("\n")
        f.close()

    except:
        print("Couldn't process file: %s" % get_single_tck_streamline_file(patient_id, stimulation_site, recording_electrode, i))


In [29]:
patient_id = 1
stimulation_site = "s00"
recording_electrode = "e01"
i = 0
streamline_id = i
single_index_file = get_fa_file(patient_id)

delete_file_if_exists(get_single_tck_streamline_file(patient_id, stimulation_site, recording_electrode, i))
delete_file_if_exists(get_single_tdi_file(patient_id, stimulation_site, recording_electrode, i))

single_streamline_list = [streamline_list[i]]

nib.streamlines.save(
    nib.streamlines.Tractogram(single_streamline_list, affine_to_rasmm=tck_file.affine), 
    get_single_tck_streamline_file(patient_id, stimulation_site, recording_electrode, i)
)    

tck2tdi = nipype.interfaces.mrtrix3.ComputeTDI(
    in_file=get_single_tck_streamline_file(patient_id, stimulation_site, recording_electrode, i),
    out_file=get_single_tdi_file(patient_id, stimulation_site, recording_electrode, i),
    reference=get_template_file(patient_id),
    terminal_output='none'
)

tck2tdi.run() 




In [42]:
nib_template_file = nib.load(get_template_file(patient_id))
dipy.tracking.utils.density_map(single_streamline_list, nib_template_file.shape, affine=nib_template_file.affine)

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

0.15394805

In [83]:
np.mod(11,10)

1